### 열 이름 변경하기

- ``rename()``: 변경 전후의 열 이름을 딕셔너리 형태로 나열
- ``inplace=True`` 옵션 설정해야 실제 반영 가능

### 열 추가
- 기존 데이터에서 계산된 결괏값을 저장하는 경우
- ``insert()`` : df.insert(몇 번째 위치, 어떤 새로운 컬럼을, 어떤 조합으로)

### 열 추가 심화
- day가 Sat, Sun 이면 1, 나머지는 0 값을 갖는 holiday 열을 추가하려면?

```py
# 0을 갖는 holiday 열을 추가
tip['holiday'] = 0

# day 값이 Sat, Sun인 경우는 holiday를 0에서 1로 변경
# df.loc[행(조건), 열]
tip.loc[ tip['day'].isin(['Sat','Sun']), 'holiday' ] = 1

# 확인
tip['holiday'].value_counts()
```

### 열 삭제

- ``drop()``
- axis=0: 행 삭제(기본 값) 
- axis=1: 열 삭제
- ``inplace=True`` 옵션을 지정해야 실제로 반영됨
- 삭제할 열을 리스트 형태로 전달해 한 번에 여러 행, 열 제거 가능

### 범줏값 변경
- ``map()``: 범주형 값을 다른 값으로 변경해줌
- 매핑되지 못한 나머지 값은 ``모두 결측치 처리`` 되므로 코드 재실행 불가능
- ``inplace 적용 불가``, 변수로 받아줘야 함
```py
# 예시
tip['sex'] = tip['sex'].map({'Male':1, 'Female':0})
```
# 
- ``replace()``: 범주형 값을 다른 값으로 변경해줌
- 매핑되지 못한 나머지 값은 ``그대로`` 남기므로 코드 재실행 가능
```py
# 예시
tip['sex'] = tip['sex'].replace({'Male':1, 'Female':0})
```

### 범줏값 만들기
- ``이산화(Discretization)``: 연속값을 구간을 나누어 범줏값으로 표현하는 과정
- ``cut() 함수``: ``크기``를 기준으로 구간을 나눌 때
- 범위 개수를 지정해주면 그에 따라 자동으로 크기를 기준으로 구분
- ``.describe()[구하고 싶은 집계값]`` = 해당 집계값 결과 출력
```py
tip['total_bill'].describe()['25%']
```
##### 예시
```py
# tip['tip'] 크기를 기준으로 4구간(a ~ d)으로 나누기
label = list('abcd')
tip['tip_grp'] = pd.cut(tip['tip'], 4, labels=label)

# 확인
tip.head()
```

- ``tip_grp`` 라는 새로운 컬럼이 생기고, (a, b] 형태로 저장
- ``(a, b]`` : a 초과 b 이하 (= b는 포함)
# 
- 각 구간의 경계 값 직접 지정 가능
```py
# 등급 구하기

# 범위 지정 (아래의 경우 기술통계 값 확인 후 사분위수 기준으로 범위 지정)
bin = [-np.inf, 2.0, 2.88, 3.60, 10.0]

# 라벨 지정
label = ['a', 'b', 'c', 'd']

# cut() 함수 활용해 새로운 컬럼 생성('tip_grp')
tip['tip_grp'] = pd.cut(tip['tip'], bins=bin, labels=label)

# 확인
tip['tip_grp'].value_counts()
```
# 
- ``qcut() 함수``: ``개수``를 기준으로 구간을 나눌 때
- 구간 개수를 지정하면 자동으로 동일한 개수를 갖는 구간들이 생김
- ``qcut()으로 4개 구간으로 나눈 결과 = 사분위수 기준으로 나눈 결과``
```py
# 같은 개수의 total_bill을 갖는 4개 구간으로 나누기
tip['bill_grp2'] = pd.qcut(tip['total_bill'], 4, labels=list('abcd'))

# 확인
tip.head()
```

### 결측치 처리
- NaN 값은 제거하거나 다른 값으로 채워야 함 (최빈값, 평균값, bfill, ffill 등)
- 필요 없을 땐 떨구고(dropna), 필요할 경우 빈 자리에 대체 값을 채워준다(fillna, interpolate)

### 결측치 찾기
1) ``info()`` 메소드로 확인
2) Non-null 앞 숫자가 전체 인덱스 개수보다 적다면, 결측치가 있는 것
3) ``isna() = isnull()``: 결측치라면 True 반환
4) ``notna() = notnull()``: 결측치라면 False 반환
5) ``sum()``: True 값의 개수, 즉 열의 결측치 개수 확인 가능
    > df.isna().sum()

### 결측치 제거
- ``dropna()``: 열이나 행 제거 가능
- ``inplace=True`` 옵션 지정해야 실제로 반영됨
- ``axis`` 옵션으로 행 제거 / 열 제거 선택

1) 결측치가 하나라도 있는 모든 행 제거
    - dropna(axis=0, inplace=True)
# 
2) 특정 열에 결측치가 있는 행 제거
    - dropna(``subset = [원하는 컬럼명]``, axis=0, inplace=True)
# 
3) 결측치가 있는 모든 열 제거
    - ``axis=1`` 옵션 지정

### 결측치 채우기
- ``fillna()``: 결측치 대체

##### 1) 평균값으로 채우기
- 결측치가 있는 열의 평균값(mean)으로 결측치 채우기

```py
# 데이터프레임 복사
air_test = air.copy()

# Ozone 평균 구하기
mean_Ozone = air_test['Ozone'].mean()

# 결측치를 평균값으로 채우기
air_test['Ozone'].fillna(mean_Ozone, inplace=True)

```

##### 2) 특정 값으로 채우기
- 모든 결측치, 또는 일부 결측치를 특정 값으로 채워줌

```py
# Solar.R 열의 누락된 값을 0으로 채우기
air_test['Solar.R'].fillna(0, inplace=True)

# 확인
air_test.isna().sum()
```

##### 3) 직전 행 또는 바로 다음 행 값으로 채우기
- 결측치를 바로 앞 혹은 바로 뒤에 나오는 값으로 채움
- 날짜 또는 시간의 흐름에 따른 값을 갖는 ``시계열 데이터`` 처리시 유용
- ``method='ffill'``: 바로 앞의 값으로 변경(Fowared Fill)
- ``method='bfill'``: 바로 다음 값으로 변경(Backwared Fill)

```py
# ffill: Ozone 열의 누락된 값을 바로 앞의 값으로 채우기
air_test['Ozone'].fillna(method='ffill', inplace=True)

# bfill: Solar.R 열의 누락된 값을 바로 뒤의 값으로 채우기
air_test['Solar.R'].fillna(method='bfill', inplace=True)

# 확인
air_test.isna().sum()
```

##### 4) 선형보간법으로 채우기
- ``interpolate()`` 메소드

```py
# 데이터프레임 복사
air_test = air.copy()

# 선형보간법으로 채우기
air_test['Ozone'].interpolate(method='linear', inplace=True)
air_test['Solar.R'].interpolate(method='linear', inplace=True)

# 확인
air_test.isna().sum()
```